In [16]:
directory = 'data/ctgov2016_2017'
import xml.etree.ElementTree as etree
import xmltodict
import sqlite3
import json
import os

In [17]:
db = sqlite3.connect('db.sqlite3')

In [18]:
def expand(root, branch):
    #print(root.tag, root.attrib)
    branch.append(root.tag)
    if len(root.attrib):
        for attribute in root.attrib:
            branch.append(attribute)
            columns.append('_'.join(branch[1:]))
            data.append([root.attrib[attribute]])
            branch.pop()
    if len(root):
        for child in root:
            expand(child, branch)
    else:
        if '_'.join(branch[1:]) in columns:
            data[columns.index('_'.join(branch[1:]))].append(root.text)
        else:
            columns.append('_'.join(branch[1:]))
            data.append([root.text])
    branch.pop()

In [20]:
cnt = 0 
for filename in os.listdir(directory):
    tree = etree.parse(os.path.join(directory, filename))
    root = tree.getroot()
    data = []
    columns = []
    branch = []
    expand(root, branch)
    for i,datum in enumerate(data):
        if len(datum) < 2:
            data[i] = datum[0]
        else:
            data[i] = repr(datum)
    keys = tuple(data)
    if cnt == 0:
        query = "create table if not exists ctgov({0}{1})"
        query = query.format(" text,".join(columns), " text")
        c = db.cursor()
        c.execute(query)
        c.close()
    else:
        c = db.cursor()
        c.execute("PRAGMA table_info(ctgov)")
        cols = [p[1] for p in c.fetchall()]
        for col in columns:
            if col not in cols:
                query = "alter table ctgov add column {0}{1}"
                query = query.format(col, " text;")
                c.execute(query)
        c.close()
    
    query = "insert into ctgov ({0}) values (?{1})"
    query = query.format(",".join(columns), ",?" * (len(columns)-1))
    c = db.cursor()
    c.execute(query, keys)
    c.close()
    cnt += 1

KeyboardInterrupt: 

In [ ]:
db.commit()

In [ ]:
db.close()

In [ ]:
query

In [ ]:
keys